In [1]:
import torch
import typing
import functorch

# 2.4 Broadcasting

In [ ]:
a = [3]
b = [4]
c = [5]

Xa = torch.rand(a)
# F: a -> b
def F(Xa: torch.Tensor) -> torch.Tensor:
    return sum(Xa**2)

# This broadcasts F over an arbitrary size.
# F: * a -> * b
Fxc = functorch.vmap(F)

# Tensoring shapes uses the + symbol.
Xac = torch.rand(a + c)

# We can show Definition 2.13 by,
for i in range(c[0]):
    assert F(Xac[:,i])==Fxc(Xac)[:,i]

def Fxc1(Xac: torch.Tensor) -> torch.Tensor:
    Xbc = torch.zeros()